In [ ]:
# Set openai key
# Please gen and add `OPENAI_API_KEY` in the `.env` file in the current directory
# https://platform.openai.com/account/api-keys
# Please gen and add `SERPAPI_API_KEY` in the `.env` file in the current directory
# https://serpapi.com/dashboard
# Create a new project at https://console.developers.google.com/apis/dashboard
# Create a new API key at https://console.developers.google.com/apis/credentials
# Enable the Custom Search API at https://console.developers.google.com/apis/library/customsearch.googleapis.com
# Create a new Custom Search Engine at https://cse.google.com/cse/all
# Add your API Key as `GCS_DEVELOPER_KEY` and your Custom Search Engine ID as `GCS_CX` to your .env file
import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
LLM_MODEL = "gpt-3.5-turbo"

## Simple tools

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=1, model=LLM_MODEL)

In [ ]:
# Without wiki tool
print(llm.invoke("Why did the Whisky War between Denmark and Canada start?").content)

In [ ]:
# With wiki tool
from langchain.agents import initialize_agent, load_tools


tools = load_tools(["wikipedia",], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    handle_parsing_errors=True,
)

In [ ]:
# https://en.wikipedia.org/wiki/Whisky_War
agent.run("Why did the Whisky War between Denmark and Canada start?")

## SerpAPI Tool

In [ ]:
from langchain.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent

In [ ]:
search = SerpAPIWrapper()

# Creating tool via `Tool` class
serpapi_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to find tweets",
)

serp_api_agent = initialize_agent(
    [serpapi_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_execution_time=10000,
)

In [ ]:
serp_api_agent.run("What is the most popular tweet ever?")

## Custom tools

In [ ]:
from langchain.agents import tool
from datetime import date

# Creating tool via decorator `tool`. Docstring is used for descrition of a tool
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any questions related to knowing todays date.
    This function will always return todays date - any date mathmatics should occur outside this function.
    """
    return date.today().strftime("%d %B, %Y")

In [ ]:
time_agent = initialize_agent(
    [time],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_execution_time=10000,
)

time_agent.run("Date of today")

In [ ]:
from langchain.agents import tool
from IPython.display import Image, display
from googleapiclient.discovery import build

@tool
def show_image_of(dish: str) -> str:
    """
    Use it to show an image of the provided dish.
    This function has an input parameter `dish`.
    According to this parameter, the function will try to find and display an image.
    The function returns True if an image of the requested dish was displayed and False otherwise.
    """
    try:
        service = build(
            "customsearch", "v1", developerKey=os.environ["GCS_DEVELOPER_KEY"]
        )

        search_res = (
            service.cse()
            .list(
                q=dish,
                cx=os.environ["GCS_CX"],
                searchType='image',
                num=1,
            )
            .execute()
        )
        url = search_res["items"][0]["link"]
        display(Image(url=url))
    except Exception as err:
        print(err)
        return False
    return True

In [ ]:
image_agent = initialize_agent(
    [show_image_of],
    llm,
    max_execution_time=10000,
)

In [ ]:
image_agent.run("Provide information about pizza and display photo")

### Using agent in a chain

In [ ]:
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model=LLM_MODEL)

In [ ]:
prompt1 = ChatPromptTemplate.from_template(
    """
    Provide the most popular meal of {country}.
    The answer must contain ONLY the dish.
    """
)

chain1 = prompt1 | llm

In [ ]:
print(chain1.invoke({"country": "Itali"}).content)

In [ ]:
import requests
from bs4 import BeautifulSoup

@tool
def provide_recipe_of(dish: str) -> str:
    """
    Use it to find the recipe for a dish.
    This function has an input parameter `dish`.
    According to this parameter, the function will try to find the recipe of the requested meal and return recipe as a string.
    The parsing of recipe, summarization, and formatting (like making step-by-step instructions) must be done beyond this function.
    The function returns an empty string if the recipe is not found.
    """
    try:
        service = build(
            "customsearch", "v1", developerKey=os.environ["GCS_DEVELOPER_KEY"]
        )

        search_res = (
            service.cse()
            .list(
                q=f"{dish} recipe",
                cx=os.environ["GCS_CX"],
                num=10,
            )
            .execute()
        )

        for item in search_res["items"]:
            response = requests.get(item["link"])
            if response.status_code == 200:
                break
        else:
            return ""

        bs = BeautifulSoup(response.text, features="html.parser")

        recipe = "\n".join(line for line in bs.get_text().split("\n") if len(line)).lower()
    
    except Exception as err:
        return ""
    else:
        return recipe[5000:15000]


dish_agent = initialize_agent(
    [
        *load_tools(["wikipedia",], llm=llm),
        show_image_of,
        provide_recipe_of,
    ],
    llm,
    handle_parsing_errors=True,
)

prompt2 = ChatPromptTemplate.from_template(
    """
    Provide information from wiki, step-by-step recipe, and display a photo of the following meal: {meal}.
    """
)

chain2 = {"meal": chain1} | prompt2 | dish_agent

In [ ]:
print(chain2.invoke({"country": "Itali"})["output"])